This script shows how data from the different model simulation runs and countries is aggregated to country and region level. The example is only done for Latin America and a small model selection of 6 GHM and climate forcing combinations. Real data contains all the countries and 46 model combinateions.
In order to see how damages are generated with CLIMADA to create the input file please the the CLIMADA RiverFlood tutorial here:

https://github.com/CLIMADA-project/climada_python/blob/main/doc/tutorial/climada_hazard_RiverFlood.ipynb

The pre-processing that is additionally necessary to convert asset damages and observed damages is described in the readme.txt under chapter 4 Instructions for use.

In [4]:
import numpy as np
import pandas as pd
import os
# load data (adjust path)
ass_data=pd.read_csv('/home/insauer/projects/NC_Submission/Climada_papers/climada_papers/202010_flood_attribution/Demo/Demo_Data/DEMO_AssembledDataSubregions.csv')
ass_data[ass_data['D_CliExp_Pos']>0]

,Unnamed: 0,Year,Country,Region,GHM,clim_forc,D_CliExp_Pos,D_CliExp_Neg,D_1980_Pos,D_1980_Neg,D_2010_Pos,D_2010_Neg,D_obs_dummy_Pos,D_obs_dummy_Neg
18,369,1971,ARG,LAM,orchidee,wfdei,1.096388e+06,6.456917e+07,1.669235e+06,1.029778e+08,9.511688e+06,5.980761e+08,NaN,NaN
21,398,1971,ARG,LAM,lpjml,wfdei,8.936264e+05,1.306591e+07,1.359166e+06,2.009086e+07,7.811199e+06,1.204149e+08,NaN,NaN
23,409,1971,ARG,LAM,matsiro,wfdei,3.021842e+05,8.957474e+07,4.560491e+05,1.406277e+08,2.550091e+06,7.986013e+08,NaN,NaN
31,1019,1971,BHS,LAM,lpjml,princeton,3.452119e-02,0.000000e+00,6.223493e-02,0.000000e+00,1.729590e-01,0.000000e+00,NaN,NaN
33,1042,1971,BHS,LAM,lpjml,wfdei,1.872020e-03,0.000000e+00,3.374885e-03,0.000000e+00,9.379248e-03,0.000000e+00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11263,396569,2010,VEN,LAM,lpjml,princeton,2.455575e+09,1.848530e+09,1.133333e+09,1.013319e+09,2.455575e+09,1.848530e+09,1.227788e+08,9.242652e+07
11264,396582,2010,VEN,LAM,orchidee,princeton,2.991682e+09,1.354734e+09,1.436558e+09,7.402328e+08,2.991682e+09,1.354734e+09,1.495841e+08,6.773672e+07
11265,396586,2010,VEN,LAM,lpjml,wfdei,1.990408e+09,8.089158e+07,8.697089e+08,4.454031e+07,1.990408e+09,8.089158e+07,9.952041e+07,4.044579e+06
11266,396591,2010,VEN,LAM,matsiro,princeton,1.003280e+09,1.635081e+09,4.573970e+08,8.996332e+08,1.003280e+09,1.635081e+09,5.016400e+07,8.175404e+07


In [ ]:
def aggregation_regions(x):
    """
    This function aggregates country-level damages and variables to
    regional level.
    Parameters
    ----------
    x : DataFrame
        country-level damages and other indicators for all model combinations

    Returns
    -------
    DataFrame
        regionally aggregated damages and other indicators
    """
    aggregated_model_damages_pos = x['D_CliExp_Pos'].sum()
    aggregated_model_damages_neg = x['D_CliExp_Neg'].sum()
    aggregated_model_damages_1980_pos = x['D_1980_Pos'].sum()
    aggregated_model_damages_1980_neg = x['D_1980_Neg'].sum()
    aggregated_model_damages_2010_pos = x['D_2010_Pos'].sum()
    aggregated_model_damages_2010_neg = x['D_2010_Neg'].sum()
    aggregated_observed_damages_pos = (x['D_obs_dummy_Pos']).sum()
    aggregated_observed_damages_neg = (x['D_obs_dummy_Neg']).sum()
    return pd.Series([aggregated_model_damages_pos,
                      aggregated_model_damages_neg,
                      aggregated_model_damages_1980_pos,
                      aggregated_model_damages_1980_neg,
                      aggregated_model_damages_2010_pos,
                      aggregated_model_damages_2010_neg,
                      aggregated_observed_damages_pos,
                      aggregated_observed_damages_neg],
                     index=['D_CliExp_Pos', 'D_CliExp_Neg',
                            'D_1980_Pos', 'D_1980_Neg',
                            'D_2010_Pos', 'D_2010_Neg',
                            'D_obs_dummy_Pos',
                            'D_obs_dummy_Pos'])

In [5]:
def region_aggregation(cols, dataFrame):
    """
    This function is a wrapper for the aggregation and selects the columns to
    be aggregated to regional level.

    Parameters
    ----------
    cols : string list
        Columns to be aggregated

    Returns
    -------
    DataFrame
         regionally aggregated damages and other indicators regions
    """
    data_region = dataFrame.groupby(['Year', 'GHM', 'cli_forc', 'Region'])\
                                     [out_cols].apply(aggregation_regions)\
                                     .reset_index()  # groupby year and model

    return data_region

In [ ]:
def func_median(x):
    """
    This function aggregates the damages and other indicators from the
    different model runs to the model median and adds basic statistics such as
    the one-third and two-third quantiles.

    Parameters
    ----------
    x : DataFrame
        regionally aggregated damages and other indicators for all model
        combinations

    Returns
    -------
    DataFrame
         model medians of regionally aggregated damages and other indicators
    """
    # identify the median of the model data:
    median_model_damages_pos = x['D_CliExp_Pos'].median()  # =quantile(0.5)
    median_model_damages_neg = x['D_CliExp_Neg'].median()
    median_model_damages_1980_pos = x['D_1980_Pos'].median()  # =quantile(0.5)
    median_model_damages_1980_neg = x['D_1980_Neg'].median()
    median_model_damages_2010_pos = x['D_2010_Pos'].median()  # =quantile(0.5)
    median_model_damages_2010_neg = x['D_2010_Neg'].median()
    median_observed_damages_pos = (x['D_obs_dummy_Pos']).mean()  # all the same value
    median_observed_damages_neg = (x['D_obs_dummy_Neg']).mean()  # all the same value
    one_third_quantile_model_damages_pos = x['D_CliExp_Pos'].quantile(0.3)  # 30
    two_third_quantile_model_damages_pos = x['D_CliExp_Pos'].quantile(0.7)
    one_third_quantile_model_damages_neg = x['D_CliExp_Neg'].quantile(0.3)  # 30
    two_third_quantile_model_damages_neg = x['D_CliExp_Neg'].quantile(0.7)
    one_third_quantile_model_damages_1980_pos = x['D_1980_Pos'].quantile(0.3)  # 30
    two_third_quantile_model_damages_1980_pos = x['D_1980_Pos'].quantile(0.7)  # 70
    one_third_quantile_model_damages_1980_neg = x['D_1980_Neg'].quantile(0.3)  # 30
    two_third_quantile_model_damages_1980_neg = x['D_1980_Neg'].quantile(0.7)  # 70

    one_third_quantile_model_damages_2010_pos = x['D_2010_Pos'].quantile(0.3)  # 30
    two_third_quantile_model_damages_2010_pos = x['D_2010_Pos'].quantile(0.7)  # 70
    one_third_quantile_model_damages_2010_neg = x['D_2010_Neg'].quantile(0.3)  # 30
    two_third_quantile_model_damages_2010_neg = x['D_2010_Neg'].quantile(0.7)  # 70

    # flood_vol = x['FloodVol_Flopros'].median()
    return pd.Series([median_model_damages_pos,
                      median_model_damages_neg,
                      median_model_damages_1980_pos,
                      median_model_damages_1980_neg,
                      median_model_damages_2010_pos,
                      median_model_damages_2010_neg,
                      median_observed_damages_pos,
                      median_observed_damages_neg,
                      one_third_quantile_model_damages_pos,
                      two_third_quantile_model_damages_pos,
                      one_third_quantile_model_damages_neg,
                      two_third_quantile_model_damages_neg,
                      one_third_quantile_model_damages_1980_pos,
                      two_third_quantile_model_damages_1980_pos,
                      one_third_quantile_model_damages_1980_neg,
                      two_third_quantile_model_damages_1980_neg,
                      one_third_quantile_model_damages_2010_pos,
                      two_third_quantile_model_damages_2010_pos,
                      one_third_quantile_model_damages_2010_neg,
                      two_third_quantile_model_damages_2010_neg],
                     index=['D_CliExp_Pos',
                            'D_CliExp_Neg',
                            'D_1980_Pos',
                            'D_1980_Neg',
                            'D_2010_Pos',
                            'D_2010_Neg',
                            'D_obs_dummy_Pos',
                            'D_obs_dummy_Neg',
                            'D_CliExp_Pos_onethird_quantile',
                            'D_CliExp_Pos_twothird_quantile',
                            'D_CliExp_Neg_onethird_quantile',
                            'D_CliExp_Neg_twothird_quantile',
                            'D_1980_Pos_onethird_quantile',
                            'D_1980_Pos_twothird_quantile',
                            'D_1980_Neg_onethird_quantile',
                            'D_1980_Neg_twothird_quantile',
                            'D_2010_Pos_onethird_quantile',
                            'D_2010_Pos_twothird_quantile',
                            'D_2010_Neg_onethird_quantile',
                            'D_2010_Neg_twothird_quantile'])